In [2]:
# print('Program started')
import vrep
import numpy as np
import cv2
import cv2.aruco as aruco
import sys, time, math
from platform import python_version

##############Comunicação com V-REP##########################
serverIP = '127.0.0.1';
serverPort = 19999; #Esta porta do servidor está sempre aberta
vrep.simxFinish(-1);
clientID=vrep.simxStart(serverIP,serverPort,True,True,5000,5);
#############################################################

windowName = "Imagem-Processada" #Name of the window created
cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)#Setting the name ande type of window
cv2.setWindowProperty(windowName, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FREERATIO)#setting fullscreen

#-- Define Tag
id_to_find = 2
marker_size = 100 #-cm

#-- Get the camera calibration
calib_path = ''
camera_matrix = np.loadtxt(calib_path+'cameraMatrix.txt', delimiter = ',')
camera_distortion = np.loadtxt(calib_path+'cameraDistortion.txt', delimiter = ',')

#-- 180 deg rotation matrix around x axis
R_flip = np.zeros((3,3), dtype=np.float)
R_flip[0,0] = 1.0
R_flip[1,1] = -1.0
R_flip[2,2] = -1.0

ret = np.zeros((3,3), dtype=np.float)

#-- Define the Aruco dictionary
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_50)
parameters =  aruco.DetectorParameters_create()

if clientID!=-1:
    print ('Connected to remote API server')
    print ('Versão OpenCV: ',cv2.__version__)
    print ('Versão Python: ',python_version())
    
    err,visionHandle = vrep.simxGetObjectHandle(clientID,'Vision_sensor',vrep.simx_opmode_oneshot_wait)
    
    while err == 0:
        
        err,visionHandle = vrep.simxGetObjectHandle(clientID,'Vision_sensor',vrep.simx_opmode_oneshot_wait)
        err,res,imgList = vrep.simxGetVisionSensorImage(clientID,visionHandle,0,vrep.simx_opmode_oneshot_wait) #Imagens do sensor
        
        img = (np.array(imgList))[::-1]#Passa de lista para uma array e inverte a array para plotar corretamente
        imgRGB = cv2.flip(np.uint8(img.reshape(720,1280,3)),1) #Transforma em uma imagem RGB e espelha
        
        #-- Convert in gray scale
        gray = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2GRAY) #-- remember, OpenCV stores color images in Blue, Green, Red
        
        #-- Find all the aruco markers in the image
        corners, ids, rejected = aruco.detectMarkers(image=gray, 
                                                     dictionary=aruco_dict, 
                                                     parameters=parameters,
                                                     cameraMatrix=camera_matrix, 
                                                     distCoeff=camera_distortion)
        
        if ids != None and ids[0] == id_to_find:
        
            #-- ret= [rvec,tvec, ?]
            #-- array of rotation and position of each marker in camera frame
            #-- rvec = [[rvec_1, [rvec2], ...]]  attitude of the marker respect to camera frame
            #-- tvec = [[tvec_1, [tvec2], ...]]  position of the marker in camera frame
            ret = aruco.estimatePoseSingleMarkers(corners, marker_size, camera_matrix, camera_distortion)

            #-- Unpack the output, get only the first
            rvec, tvec = ret[0][0,0,:], ret[1][0,0,:]

            #-- Draw the detected marker and put a reference frame over it
            aruco.drawDetectedMarkers(imgRGB, corners)
            aruco.drawAxis(imgRGB, camera_matrix, camera_distortion, rvec, tvec, 40)

            #Display the resulting frame
            cv2.imshow(windowName,imgRGB)

            if cv2.waitKey(1) == 27: # exit on ESC
                break
        
        else:
            print("Nothing detected!")
            #Display the resulting frame
            cv2.imshow(windowName,imgRGB)          
            if cv2.waitKey(1) == 27: # exit on ESC
                break

    # Now close the connection to V-REP:
    vrep.simxFinish(clientID)

else:
    print ('Failed connecting to remote API server')
print ('Program ended')

cv2.destroyAllWindows()

Connected to remote API server
Versão OpenCV:  3.4.2
Versão Python:  3.6.5
Nothing detected!
Nothing detected!
Nothing detected!
Program ended
